In [2]:
# Here we take care of paths.

from pathlib import Path
import os
print('Starting path:' + os.getcwd())
if os.getcwd()[-18:] == 'VESUVIUS_Challenge':
    pass
else:
    PATH = Path().resolve().parents[0]
    os.chdir(PATH)

# make sure you are in Paragraph_to_Tex folder
print('Current path:' + os.getcwd())

Starting path:/Users/gregory/PROJECT_ML/VESUVIUS_Challenge
Current path:/Users/gregory/PROJECT_ML


In [2]:
import monai
import torch
import matplotlib.pyplot as plt
import tempfile
import shutil
import os
import glob
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from Data_Modules.MONAI_Dict_Dataset_Module import MONAI_CSV_Scrolls_Dataset
import matplotlib.patches as patches
from lit_models.UNET_monai_lit import UNET_lit
from monai.visualize import matshow3d

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

2023-04-18 19:14:49,687 - Created a temporary directory at /var/folders/wc/60y8v25x3ns_jgsx6clbdb180000gn/T/tmpuacy9zda
2023-04-18 19:14:49,687 - Writing /var/folders/wc/60y8v25x3ns_jgsx6clbdb180000gn/T/tmpuacy9zda/_remote_module_non_scriptable.py


In [3]:
KAGGLE_DIR = PATH / "kaggle"

INPUT_DIR = KAGGLE_DIR / "input"

COMPETITION_DATA_DIR = INPUT_DIR / "vesuvius-challenge-ink-detection"

TRAIN_DATA_CSV_PATH = COMPETITION_DATA_DIR / "data_train_0.7.csv"
TEST_DATA_CSV_PATH = COMPETITION_DATA_DIR / "data_test_0.7.csv"

In [4]:
dataset = MONAI_CSV_Scrolls_Dataset(
    z_dim=30,
    batch_size=1,
    data_csv_path=TRAIN_DATA_CSV_PATH,
    num_workers=0,
    num_samples=1,
    patch_size=(256,256),
    val_fragment_id=3,
    
)


In [5]:
lit_model = UNET_lit( 
        z_dim = 30,
        patch_size = (512,512),
        sw_batch_size=16 ,
        eta_min = 1e-6,
        max_epochs = 700,
        weight_decay = 0.00005,
        learning_rate = 0.0003,
        gamma = 0.85,)

In [6]:
trainer = pl.Trainer(
        accelerator='mps',
        #benchmark=True,
        max_epochs=700,
        check_val_every_n_epoch= 1,
        devices=1,
        #fast_dev_run=fast_dev_run,
        logger=pl.loggers.CSVLogger(save_dir='logs/'),
        log_every_n_steps=1,
        
        overfit_batches=0,
        precision=16,
         accumulate_grad_batches=1,)

trainer.fit(lit_model, datamodule=dataset)

2023-04-18 19:14:49,861 - Using 16bit Automatic Mixed Precision (AMP)
2023-04-18 19:14:49,870 - GPU available: True (mps), used: True
2023-04-18 19:14:49,871 - TPU available: False, using: 0 TPU cores
2023-04-18 19:14:49,871 - IPU available: False, using: 0 IPUs
2023-04-18 19:14:49,871 - HPU available: False, using: 0 HPUs
# train: 2
# val: 1
2023-04-18 19:14:49,978 - 
  | Name    | Type       | Params
---------------------------------------
0 | metrics | ModuleDict | 0     
1 | model   | UNet       | 1.6 M 
2 | loss    | MaskedLoss | 0     
---------------------------------------
1.6 M     Trainable params
0         Non-trainable params
1.6 M     Total params
6.533     Total estimated model params size (MB)


/Users/gregory/anaconda3/lib/python3.10/site-packages/lightning_fabric/connector.py:562: UserWarning: 16 is supported for historical reasons but its usage is discouraged. Please set your precision to 16-mixed instead!
  rank_zero_warn(
/Users/gregory/anaconda3/lib/python3.10/site-packages/torch/cuda/amp/grad_scaler.py:120: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.")
/Users/gregory/anaconda3/lib/python3.10/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:612: UserWarning: Checkpoint directory logs/lightning_logs/version_0/checkpoints exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")


RuntimeError: Error while merging hparams: the keys ['patch_size'] are present in both the LightningModule's and LightningDataModule's hparams but have different values.